In [14]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.features import rasterize

# Path to the GSHHS coastline shapefile (low resolution)
coastline_shapefile = 'gshhg-shp-2.3.7/GSHHS_shp/l/GSHHS_l_L1.shp'

# Path to your capacity factor raster from Global Wind Atlas
raster_path = 'FRA_capacity-factor_IEC1.tif'

# Load the raster data
with rasterio.open(raster_path) as src:
    raster_data = src.read(1)  # First band
    transform = src.transform
    shape = src.shape
    crs = src.crs  # Coordinate reference system

# Load coastline shapefile
coastline_gdf = gpd.read_file(coastline_shapefile)

# Reproject coastline data to match the raster CRS
coastline_gdf = coastline_gdf.to_crs(crs)

# Rasterize the coastline data
land_sea_mask = rasterize(
    [(geom, 1) for geom in coastline_gdf.geometry],  # Land is 1
    out_shape=shape,
    transform=transform,
    fill=0,  # Sea is 0
    dtype='uint8'
)

# Separate land and sea regions
land_mask = land_sea_mask == 1  # Land
sea_mask = land_sea_mask == 0  # Sea

# Calculate area-weighted capacity factors
land_values = raster_data[land_mask]
sea_values = raster_data[sea_mask]

land_mean = np.nanmean(land_values) if np.any(land_mask) else np.nan
sea_mean = np.nanmean(sea_values) if np.any(sea_mask) else np.nan

print(f"Area-weighted capacity factor on land: {land_mean}")
print(f"Area-weighted capacity factor on sea: {sea_mean}")

Area-weighted capacity factor on land: 0.3083297610282898
Area-weighted capacity factor on sea: 0.46371522545814514


In [2]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask
from shapely.geometry import mapping
from rasterio.features import rasterize
import pandas as pd

# Paths to input data
raster_path = 'gwa3_250_capacityfactor_IEC1.tif'  # Global raster file
countries_shapefile = '/Users/romain/GitHub/premise-johanna/dev/ne_110m_admin_0_countries/ne_110m_admin_0_countries.shp'  # Country boundaries
coastline_shapefile = '/Users/romain/GitHub/premise-johanna/dev/gshhg-shp-2.3.7/GSHHS_shp/l/GSHHS_l_L1.shp'

# Load the country and coastline shapefiles
countries_gdf = gpd.read_file(countries_shapefile)
coastline_gdf = gpd.read_file(coastline_shapefile)

# Initialize results list
results = []

# Open the raster data (keep it open throughout the loop)
with rasterio.open(raster_path) as src:
    raster_data = src.read(1)  # First band
    profile = src.profile
    transform = src.transform
    crs = src.crs
    nodata = profile.get('nodata', np.nan)

    # Reproject country and coastline data to match the raster CRS
    countries_gdf = countries_gdf.to_crs(crs)
    coastline_gdf = coastline_gdf.to_crs(crs)

    # Rasterize the coastline shapefile to create a land-sea mask
    land_sea_mask = rasterize(
        [(geom, 1) for geom in coastline_gdf.geometry],  # Land=1
        out_shape=raster_data.shape,
        transform=transform,
        fill=0,  # Sea=0
        dtype='uint8'
    )

    # Iterate over each country
    for _, country in countries_gdf.iterrows():
        country_name = country['ADMIN']  # Change 'NAME' to the column with country names
        geometry = [mapping(country['geometry'])]
        

        # Mask the raster with the country's geometry
        try:
            out_image, out_transform = mask(src, geometry, crop=True)
            masked_data = out_image[0]

            # Replace no-data values with NaN
            masked_data = np.where(masked_data == nodata, np.nan, masked_data)

            # Create a mask for land and sea regions within the country
            mask_land_sea = rasterize(
                [(geom, 1) for geom in coastline_gdf.geometry],
                out_shape=out_image.shape[1:],  # Match cropped raster
                transform=out_transform,
                fill=0,
                dtype='uint8'
            )

            # Separate onshore and offshore
            onshore_data = masked_data[mask_land_sea == 1]
            offshore_data = masked_data[mask_land_sea == 0]

            # Compute averages for onshore and offshore
            onshore_mean = np.nanmean(onshore_data) if onshore_data.size > 0 else np.nan
            offshore_mean = np.nanmean(offshore_data) if offshore_data.size > 0 else np.nan

            # Append results
            results.append({
                'Country': country_name,
                'Onshore Mean': onshore_mean,
                'Offshore Mean': offshore_mean
            })

            print(country_name, onshore_mean, offshore_mean)

        except Exception as e:
            print(f"Error processing {country_name}: {e}")
            results.append({
                'Country': country_name,
                'Onshore Mean': np.nan,
                'Offshore Mean': np.nan
            })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save the results to a CSV file
results_df.to_csv('country_onshore_offshore_averages.csv', index=False)

# Display the DataFrame
print(results_df)


Fiji 0.16400221 0.27551243
United Republic of Tanzania 0.11885374 0.16631524


/var/folders/cn/pys1j9rn0y7djkhv3hfdtrs00000gn/T/ipykernel_56847/2393544920.py:71: RuntimeWarning: Mean of empty slice
  offshore_mean = np.nanmean(offshore_data) if offshore_data.size > 0 else np.nan


Western Sahara 0.5310788 nan
Canada 0.33350283 0.3578078
United States of America 0.30065763 0.39959982
Kazakhstan 0.37614635 nan
Uzbekistan 0.3389081 nan
Papua New Guinea 0.051962893 0.11727087
Indonesia 0.041844975 0.08171708
Argentina 0.35616446 0.6060382
Chile 0.29946798 0.5540727
Democratic Republic of the Congo 0.038150717 0.088206984
Somalia 0.3325917 0.46123043
Kenya 0.2052059 0.27727
Sudan 0.3677476 0.42250425
Chad 0.34711292 nan
Haiti 0.15853462 0.1624473
Dominican Republic 0.15554346 0.27555984
Russia 0.27615958 0.42854053
The Bahamas 0.20661373 0.27681592
Falkland Islands 0.6525932 0.675012
Norway 0.35422918 0.35917625
Greenland 0.60211056 0.27351907


/var/folders/cn/pys1j9rn0y7djkhv3hfdtrs00000gn/T/ipykernel_56847/2393544920.py:70: RuntimeWarning: Mean of empty slice
  onshore_mean = np.nanmean(onshore_data) if onshore_data.size > 0 else np.nan
/var/folders/cn/pys1j9rn0y7djkhv3hfdtrs00000gn/T/ipykernel_56847/2393544920.py:71: RuntimeWarning: Mean of empty slice
  offshore_mean = np.nanmean(offshore_data) if offshore_data.size > 0 else np.nan


French Southern and Antarctic Lands nan nan
East Timor 0.11413051 0.13719475
South Africa 0.23027523 0.45303553
Lesotho 0.21418619 nan
Mexico 0.1407161 0.24958505
Uruguay 0.36964247 0.43724167
Brazil 0.10742907 0.2490259
Bolivia 0.14419249 nan
Peru 0.04975577 0.3657733
Colombia 0.055657882 0.14688887
Panama 0.11383208 0.1483115
Costa Rica 0.11621972 0.08683576
Nicaragua 0.1835102 0.20104927
Honduras 0.1155342 0.21706939
El Salvador 0.113370255 0.107388
Guatemala 0.08838171 0.072382174
Belize 0.08190431 0.20307897
Venezuela 0.11023462 0.38356805
Guyana 0.057786167 0.10223242
Suriname 0.050563034 0.21364732
France 0.28333843 0.41799602
Ecuador 0.052132633 0.08984087
Puerto Rico 0.1543825 0.2827469
Jamaica 0.16216725 0.24130389
Cuba 0.20765054 0.32232377
Zimbabwe 0.15265942 nan
Botswana 0.21009873 nan
Namibia 0.21745087 0.37166533
Senegal 0.21458876 0.3107283
Mali 0.32493767 nan
Mauritania 0.45209777 0.6009767
Benin 0.12663127 0.20324387
Niger 0.339325 nan
Nigeria 0.15399791 0.10952705
Ca

...

Ethiopia 0.14466125 nan
Djibouti 0.35250968 0.32753754
Somaliland 0.31686205 0.2782409
Uganda 0.065436475 nan
Rwanda 0.044832997 nan


/var/folders/cn/pys1j9rn0y7djkhv3hfdtrs00000gn/T/ipykernel_56847/2393544920.py:71: RuntimeWarning: Mean of empty slice
  offshore_mean = np.nanmean(offshore_data) if offshore_data.size > 0 else np.nan


Bosnia and Herzegovina 0.17227586 nan
North Macedonia 0.15024514 nan


/var/folders/cn/pys1j9rn0y7djkhv3hfdtrs00000gn/T/ipykernel_56847/2393544920.py:71: RuntimeWarning: Mean of empty slice
  offshore_mean = np.nanmean(offshore_data) if offshore_data.size > 0 else np.nan


Republic of Serbia 0.2145368 nan
Montenegro 0.18365069 0.14221762
Kosovo 0.18210481 nan
Trinidad and Tobago 0.115842685 0.2417137
South Sudan 0.11593931 nan
                         Country  Onshore Mean  Offshore Mean
0                           Fiji      0.164002       0.275512
1    United Republic of Tanzania      0.118854       0.166315
2                 Western Sahara      0.531079            NaN
3                         Canada      0.333503       0.357808
4       United States of America      0.300658       0.399600
..                           ...           ...            ...
172           Republic of Serbia      0.214537            NaN
173                   Montenegro      0.183651       0.142218
174                       Kosovo      0.182105            NaN
175          Trinidad and Tobago      0.115843       0.241714
176                  South Sudan      0.115939            NaN

[177 rows x 3 columns]


In [6]:
import rasterio

# Path to the raster file
raster_path = 'gwa3_250_capacityfactor_IEC1.tif'

# Open the raster file
with rasterio.open(raster_path) as src:
    # Print general information about the raster
    print("Raster Metadata:")
    print(src.meta)
    
    # List the number of bands
    print(f"\nNumber of Bands: {src.count}")
    
    # Print CRS (coordinate reference system)
    print(f"\nCRS: {src.crs}")
    
    # Print bounds
    print(f"\nBounds: {src.bounds}")
    
    # Print resolution
    print(f"\nResolution: {src.res}")


Raster Metadata:
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -999.0, 'width': 144263, 'height': 55933, 'count': 1, 'crs': CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]'), 'transform': Affine(0.002500000000000124, 0.0, -180.33092731781537,
       0.0, -0.002500000000000124, 81.5107687477372)}

Number of Bands: 1

CRS: EPSG:4326

Bounds: BoundingBox(left=-180.33092731781537, bottom=-58.32173125226974, right=180.3265726822025, top=81.5107687477372)

Resolution: (0.002500000000000124, 0.002500000000000124)


In [ ]:
import numpy as np

# Open the raster
with rasterio.open('gwa3_250_capacityfactor_IEC1.tif') as src:
    raster_data = src.read(1)  # Read the first band
    nodata_value = src.nodata  # This should be -999.0

    # Mask no-data values
    raster_data = np.where(raster_data == nodata_value, np.nan, raster_data)

    # Check statistics ignoring no-data values
    min_value = np.nanmin(raster_data)
    max_value = np.nanmax(raster_data)
    mean_value = np.nanmean(raster_data)

    print(f"Min Value (excluding -999): {min_value}")
    print(f"Max Value (excluding -999): {max_value}")
    print(f"Mean Value (excluding -999): {mean_value}")
